In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
matplotlib.rcParams['svg.fonttype'] = 'none' # for svgs
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from scipy import stats
from pathlib import Path

import flexiznam as flz
from cottage_analysis.analysis import spheres, common_utils
from cottage_analysis.pipelines import pipeline_utils
from cottage_analysis.summary_analysis import summary_utils, get_session_list, depth_decoder_stats
from cottage_analysis.plotting import rsof_plots, depth_selectivity_plots, depth_decoder_plots
from tqdm import tqdm

In [ ]:
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)
READ_VERSION = 10
VERSION = 13
# READ_ROOT = flz.get_data_root("processed", flexilims_session=flexilims_session) / "v1_manuscript_figures"/f"ver{READ_VERSION}"
READ_ROOT = Path(
    f"/nemo/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver{READ_VERSION}"
)
SAVE_ROOT = flz.get_data_root("processed", flexilims_session=flexilims_session) / "v1_manuscript_figures"/f"ver{VERSION}"
SAVE_ROOT.mkdir(parents=True, exist_ok=True)
(SAVE_ROOT/"rsof_additional_analysis").mkdir(parents=True, exist_ok=True)

reload = False # set to True to reload everything

In [ ]:
# Concatenate all sessions for closedloop RS-OF integration data
models = ["gof", "grs", "gadd", "g2d", "gratio"]
cols = [
    "roi",
    "best_depth",
    "preferred_depth_closedloop",
    "preferred_depth_closedloop_crossval",
    "depth_tuning_popt_closedloop",
    "depth_tuning_test_rsq_closedloop",
    "depth_tuning_test_spearmanr_pval_closedloop",
    "depth_tuning_test_spearmanr_rval_closedloop",
    "preferred_RS_closedloop_g2d",
    "preferred_RS_closedloop_crossval_g2d",
    "preferred_OF_closedloop_g2d",
    "preferred_OF_closedloop_crossval_g2d",
]
cols_to_add = [
    "rsof_test_rsq_closedloop_",
    "rsof_rsq_closedloop_",
    "rsof_popt_closedloop_",
]
for model in models:
    for col in cols_to_add:
        cols.append(f"{col}{model}")

mouse_list = flz.get_entities("mouse", flexilims_session=flexilims_session)
mouse_list = mouse_list[mouse_list.name.isin(["PZAH6.4b",
                "PZAG3.4f",
                "PZAH8.2h",
                "PZAH8.2i",
                "PZAH8.2f",
                "PZAH10.2d",
                "PZAH10.2f"])]
if reload:
    results_all = summary_utils.concatenate_all_neurons_df(
        flexilims_session=flexilims_session,
        session_list=get_session_list.get_sessions(
            flexilims_session,
            exclude_openloop=False,
            exclude_pure_closedloop=False,
            v1_only=True,
            trialnum_min=10,
            mouse_list=mouse_list,
        ),
        filename="neurons_df.pickle",
        cols=cols,
        read_iscell=True,
        verbose=True,
    )
    results_all["preferred_RS_closedloop_crossval_g2d"] = (
        results_all["preferred_RS_closedloop_crossval_g2d"] * 100
    )
    results_all["preferred_depth_closedloop_crossval"] = (
        results_all["preferred_depth_closedloop_crossval"] * 100
    )
    results_all["preferred_OF_closedloop_crossval_g2d"] = np.degrees(
        results_all["preferred_OF_closedloop_crossval_g2d"]
    )
    results_all.to_pickle(SAVE_ROOT / "fig2" / "results_all_rsof_closedloop.pickle")
else:
    results_all = pd.read_pickle(READ_ROOT / "fig2" /  "results_all_rsof_closedloop.pickle")

## Is running speed tuning monotonic?

In [ ]:
example_session = "PZAH6.4b_S20220524"
project = "hey2_3d-vision_foodres_20220101"
session_name = example_session
flexilims_session = flz.get_flexilims_session(project)

vs_df_example, trials_df_example = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=2,
    return_volumes=True,
)

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=None,
    conflicts="skip",
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full)


In [ ]:

this_result = results_all[(results_all.session==example_session) & (results_all.preferred_RS_closedloop_g2d>0) & (results_all.preferred_RS_closedloop_g2d<0.4)].sort_values(
    by="depth_tuning_test_rsq_closedloop", ascending=False)
rois = this_result.roi.values
this_result.head(20)


In [ ]:
# LOAD DATA
# Load an example session
EXAMPLE_ROIS = [
   121, 312, 440, 405,
]
fontsize_dict = {"title": 8, "label": 7, "tick": 5, "legend": 5}
model_labels = ["Optic\nflow", "Running\nspeed", "Additive", "Conjunctive", "RS/OF"]
results_all["preferred_depth_amplitude"] = results_all[
    "depth_tuning_popt_closedloop"
].apply(lambda x: np.exp(x[0]))

# Plot 2 example cells with depth tuning, rs tuning, of tuning, and rsof matrix
fig = plt.figure(figsize=(18 / 2.54, 21 / 2.54))
for iroi, roi in enumerate(EXAMPLE_ROIS):
    fig.add_axes([0.05, 0.88 - 0.3*iroi, 0.2, 0.15])
    depth_tuning_kwargs = dict(
        rs_thr=None,
        plot_fit=True,
        plot_smooth=False,
        linewidth=1.5,
        linecolor="royalblue",
        closed_loop=1,
        fontsize_dict=fontsize_dict,
        markersize=10,
        markeredgecolor='w',
    )
    depth_selectivity_plots.plot_depth_tuning_curve(
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        **depth_tuning_kwargs,
    )
    plt.title(f"{example_session} {roi}", fontsize=fontsize_dict["title"])

    # rsof_plots.plot_speed_tuning(
    #     fig=fig,
    #     trials_df=trials_df_example,
    #     roi=roi,
    #     is_closed_loop=1,
    #     nbins=15,
    #     which_speed="RS",
    #     speed_min=0,
    #     speed_max=1.5,
    #     speed_thr=0,
    #     smoothing_sd=1,
    #     markersize=3,
    #     linewidth=1,
    #     markeredgecolor='w',
    #     plot_x=0.35,
    #     plot_y=0.88 - 0.46*iroi,
    #     plot_width=0.2,
    #     plot_height=0.15,
    #     fontsize_dict=fontsize_dict,
    #     legend_on=False,
    # )
    # ylim = plt.gca().get_ylim()
    # plt.xlabel("Running speed (cm/s)", fontsize=fontsize_dict["label"])

    # rsof_plots.plot_speed_tuning(
    #     fig=fig,
    #     trials_df=trials_df_example,
    #     roi=roi,
    #     is_closed_loop=1,
    #     nbins=20,
    #     which_speed="OF",
    #     speed_min=0.01,
    #     speed_max=1.5,
    #     speed_thr=0.01,
    #     of_min=1e-2,
    #     of_max=1e4,
    #     smoothing_sd=1,
    #     markersize=3,
    #     linewidth=1,
    #     markeredgecolor='w',
    #     plot_x=0.6,
    #     plot_y=0.88 - 0.46*iroi,
    #     plot_width=0.2,
    #     plot_height=0.15,
    #     fontsize_dict=fontsize_dict,
    #     legend_on=True,
    #     ylim=ylim,
    # )
    # plt.gca().set_ylabel("")
    # plt.gca().set_yticklabels([])
    # plt.xlabel("Optic flow speed (degrees/s)", fontsize=fontsize_dict["label"])

    vmin, vmax = rsof_plots.plot_RS_OF_matrix(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        log_range={
            "rs_bin_log_min": 0,
            "rs_bin_log_max": 2.5,
            "rs_bin_num": 6,
            "of_bin_log_min": -1.5,
            "of_bin_log_max": 3.5,
            "of_bin_num": 11,
            "log_base": 10,
        },
        is_closed_loop=1,
        vmin=None,
        vmax=None,
        xlabel="Running speed (cm/s)",
        ylabel="Optic flow speed\n(degrees/s)",
        plot_x=0.4,
        plot_y=0.88 - 0.3*iroi,
        plot_width=0.15,
        plot_height=0.15,
        cbar_width=0.01,
        fontsize_dict=fontsize_dict,
    )
    
    # # Plot 5 model fits
    # for imodel, (model, model_label) in enumerate(zip(models, model_labels)):
    #     if imodel == 0:
    #         ylabel = "Optic flow speed (degrees/s)"
    #     else:
    #         ylabel = ""
    #     if imodel == 1:
    #         xlabel = "Running speed (cm/s)"
    #     else:
    #         xlabel = ""

    #     rsof_plots.plot_RS_OF_fit(
    #         fig=fig,
    #         neurons_df=neurons_df_example,
    #         roi=roi,
    #         model=model,
    #         model_label=model_label,
    #         min_sigma=0.25,
    #         vmin=vmin,
    #         vmax=vmax,
    #         log_range={
    #             "rs_bin_log_min": 0,
    #             "rs_bin_log_max": 2.5,
    #             "rs_bin_num": 6,
    #             "of_bin_log_min": -1.5,
    #             "of_bin_log_max": 3.5,
    #             "of_bin_num": 11,
    #             "log_base": 10,
    #         },
    #         plot_x=0.23 + 0.12*imodel,
    #         plot_y=0.63 - 0.46*iroi,
    #         plot_width=0.15,
    #         plot_height=0.15,
    #         xlabel=xlabel,
    #         ylabel=ylabel,
    #         fontsize_dict=fontsize_dict,
    #         cbar_width=None,
    #     )
    #     if imodel > 0:
    #         plt.gca().set_yticklabels([])
    #     plt.ylabel("")